In [11]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from turtle import title
import requests
from bs4 import BeautifulSoup
import csv
import time

In [12]:
data = pd.read_csv('Data.csv')
data.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


In [13]:
Output=pd.read_csv('Output Data Structure.xlsx - Sheet1.csv')
Output.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38,https://insights.blackcoffer.com/what-if-the-c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39,https://insights.blackcoffer.com/what-jobs-wil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40,https://insights.blackcoffer.com/will-machine-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41,https://insights.blackcoffer.com/will-ai-repla...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
Positive_Words = []
with open('positive-words.txt') as f:
    lines = f.readlines()
    Positive_Words=Positive_Words+lines
for i in range(len(Positive_Words)):
    Positive_Words[i]=Positive_Words[i].replace("\n","")
print(Positive_Words)

['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation', 'accolade', 'accolades', 'accommodative', 'accomodative', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accurate', 'accurately', 'achievable', 'achievement', 'achievements', 'achievible', 'acumen', 'adaptable', 'adaptive', 'adequate', 'adjustable', 'admirable', 'admirably', 'admiration', 'admire', 'admirer', 'admiring', 'admiringly', 'adorable', 'adore', 'adored', 'adorer', 'adoring', 'adoringly', 'adroit', 'adroitly', 'adulate', 'adulation', 'adulatory', 'advanced', 'advantage', 'advantageous', 'advantageously', 'advantages', 'adventuresome', 'adventurous', 'advocate', 'advocated', 'advocates', 'affability', 'affable', 'affably', 'affectation', 'affection', 'affectionate', 'affinity', 'affirm', 'affirmation', 'affirmative', 'affluence', 'affluent', 'afford', 'affordable', 'affordably', 'afordable', 'agile', 'agilely', 'agility', 'agreeable', 'ag

In [15]:
Negative_Words = []
with open('negative-words.txt','r', encoding="ISO-8859-1") as f:
    lines = f.readlines()
    Negative_Words=Negative_Words+lines
for i in range(len(Negative_Words)):
    Negative_Words[i]=Negative_Words[i].replace("\n","")
print(Negative_Words)

['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted', 'aborts', 'abrade', 'abrasive', 'abrupt', 'abruptly', 'abscond', 'absence', 'absent-minded', 'absentee', 'absurd', 'absurdity', 'absurdly', 'absurdness', 'abuse', 'abused', 'abuses', 'abusive', 'abysmal', 'abysmally', 'abyss', 'accidental', 'accost', 'accursed', 'accusation', 'accusations', 'accuse', 'accuses', 'accusing', 'accusingly', 'acerbate', 'acerbic', 'acerbically', 'ache', 'ached', 'aches', 'achey', 'aching', 'acrid', 'acridly', 'acridness', 'acrimonious', 'acrimoniously', 'acrimony', 'adamant', 'adamantly', 'addict', 'addicted', 'addicting', 'addicts', 'admonish', 'admonisher', 'admonishingly', 'admonishment', 'admonition', 'adulterate', 'adulterated', 'adulteration', 'adulterier', 'adversarial', 'adversary', 'adverse', 'adversity', 'afflict', 'affliction', 'afflictive', 'affront', 'afraid', 'aggravate', 'aggravating', 'aggravation', 'aggression', 'aggress

In [16]:
def removeHTML(data):
    data=data.replace("<p>","")
    data=data.replace("</p>","")
    data=data.replace("<strong>","")
    data=data.replace("</strong>","")
    data=data.replace("[","")
    data=data.replace("]","")
    data=data.replace("(","")
    data=data.replace(")","")
    data=data.replace("{","")
    data=data.replace("}","")
    data=data.replace(".","")
    data=data.replace(",","")
    data=data.replace("!","")
    data=data.replace("<a>","")
    data=data.replace("</a>","")
    data=data.replace(":","")
    data=data.replace("“","")
    data=data.replace("”","")
    data=data.replace("\"","")
    data=data.replace("\'","")
    data=data.replace("?","")
    data=data.replace("!","")
    return data

In [17]:
def extract(url):
  r = requests.get(url, headers=headers)
  htmlContent = r.content

  soup = BeautifulSoup(htmlContent,'html.parser')
  para = soup.find_all('p')

  return para

In [21]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

In [22]:
k=0
for url in Output['URL']:
  text=extract(url)
  words=[]
  # text=string(text).split()
  # Sentence = len(text)

  Sentence = 0
  for i in range(len(text)):
    temp = str(text[i]).split(".")
    Sentence+=len(temp)

  for i in range(len(text)):
    temp = str(text[i]).split()
    # print(temp)
    for item in temp:
      words.append(removeHTML(item))
  # print(words)
  # break

  Personal_Pronouns=["I","he","He","her","Her","his","His","him","Him","it","It","me","Me","she","She","them","Them","they","They","us","Us","we","We","you","You"]
  Personal_Pronouns_Count=0
  for item in words:
    if item in Personal_Pronouns:
      Personal_Pronouns_Count+=1

  Complex_Words=[]
  Syllable_Count_Total=0
  for item in words:
    Syllable_Count=0
    for w in item:
      if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U'):
        Syllable_Count+=1
    if Syllable_Count>=2:
      Complex_Words.append(item)
    Syllable_Count_Total+=Syllable_Count
    if "es" in item:
      Syllable_Count_Total-=1
    if "ed" in item:
      Syllable_Count_Total-=1
  
  Total_length=0
  for item in words:
    Total_length+=len(item)
  if len(words)!=0:
    Average_Word_Length=Total_length/len(words)

  Positive_Score=0
  Negative_Score=0
  for item in words:
    if item in Positive_Words:
      Positive_Score+=1
    if item in Negative_Words:
      Negative_Score+=1
  
  Polarity_Score = (Positive_Score-Negative_Score)/((Positive_Score+Negative_Score)+0.000001)
  Subjectivity_Score=(Positive_Score+Negative_Score)/(len(words)+0.000001)

  if len(words)==0:
    Average_Sentence_Length=0
    Percentage_CompleX_Words=0
    Average_Word_Length=0
    Fog_Index=0
    Syllable_Count_Total=0
    Personal_Pronouns_Count=0
    Average_Words=0
    Complex_Count=0
    Word_Count=0
  else:
    Average_Sentence_Length= len(words)/Sentence
    Percentage_CompleX_Words= len(Complex_Words)/len(words)
    Fog_Index = 0.4*(Average_Sentence_Length+Percentage_CompleX_Words)
    Average_Words_Sentence=len(words)/Sentence
    Complex_Count= len(Complex_Words)
    Word_Count = len(words)

  Output.loc[k,['POSITIVE SCORE','NEGATIVE SCORE','POLARITY SCORE','SUBJECTIVITY SCORE','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH']]=[Positive_Score,Negative_Score,Polarity_Score,Subjectivity_Score,Average_Sentence_Length,Percentage_CompleX_Words,Fog_Index,Average_Words_Sentence,Complex_Count,Word_Count,Syllable_Count_Total,Personal_Pronouns_Count,Average_Word_Length]
  k+=1

  # print(Positive_Score)
  # print(Negative_Score)
  # print(Polarity_Score)
  #print(Subjectivity_Score)
  # print(Average_Sentence_Length)
  # print(Percentage_CompleX_Words)
  # print(Fog_Index)
  # print(Average_Words_Sentence)
  # print(Complex_Count)
  # print(Word_Count)
  # print(Syllable_Count_Total)
  # print(Personal_Pronouns_Count)
  # print(Average_Word_Length)

  # break

In [23]:
Output.to_csv('Output_data.csv',index=False)
Output

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,73.0,33.0,0.377358,0.059651,16.302752,0.607766,6.764207,16.302752,1080.0,1777.0,3672.0,22.0,5.696680
1,38,https://insights.blackcoffer.com/what-if-the-c...,65.0,38.0,0.262136,0.073310,16.529412,0.491815,6.808491,16.529412,691.0,1405.0,2491.0,34.0,4.851246
2,39,https://insights.blackcoffer.com/what-jobs-wil...,72.0,35.0,0.345794,0.063464,15.327273,0.571174,6.359379,15.327273,963.0,1686.0,3357.0,24.0,5.411625
3,40,https://insights.blackcoffer.com/will-machine-...,71.0,23.0,0.510638,0.059119,14.324324,0.520755,5.938032,14.324324,828.0,1590.0,2904.0,48.0,4.853459
4,41,https://insights.blackcoffer.com/will-ai-repla...,57.0,24.0,0.407407,0.047507,15.642202,0.514956,6.462863,15.642202,878.0,1705.0,3093.0,49.0,5.055132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,29.0,27.0,0.035714,0.063134,13.044118,0.547914,5.436813,13.044118,486.0,887.0,1674.0,24.0,5.402480
110,147,https://insights.blackcoffer.com/the-future-of...,42.0,11.0,0.584906,0.048094,16.953846,0.546279,7.000050,16.953846,602.0,1102.0,2122.0,14.0,5.341198
111,148,https://insights.blackcoffer.com/big-data-anal...,30.0,44.0,-0.189189,0.065371,12.863636,0.513251,5.350755,12.863636,581.0,1132.0,2114.0,13.0,5.042403
112,149,https://insights.blackcoffer.com/business-anal...,30.0,3.0,0.818182,0.059034,15.527778,0.561717,6.435798,15.527778,314.0,559.0,1108.0,10.0,5.574240
